In [166]:
! pip install lxml
! pip install requests
! pip install beautifulsoup4

import pandas as pd
import requests
from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])
neighborhood

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [167]:
path = 'http://cocl.us/Geospatial_data'
coordinates = pd.read_csv(path)
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [180]:
merged_df = pd.merge(neighborhood, coordinates, left_on = 'Postcode', right_on = 'Postal Code', how ='left')
merged_df.drop_duplicates(['Neighbourhood'])

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN
2,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
3,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
5,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
...,...,...,...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West,M8Z,43.628841,-79.520999
282,M8Z,Etobicoke,Mimico NW,M8Z,43.628841,-79.520999
283,M8Z,Etobicoke,The Queensway West,M8Z,43.628841,-79.520999
284,M8Z,Etobicoke,Royal York South West,M8Z,43.628841,-79.520999


In [248]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [249]:
CLIENT_ID = 'JWGT31OWR0YQK1FUZGQEQSADEFRCIGLJ40ADLDNYZ3N5NWRL' # your Foursquare ID
CLIENT_SECRET = 'I4CJIWGO5QRUEF5YLMR303DPEEICUS2OEDEGEQAGDY4ZDC1C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

merged_df.loc[4, 'Borough']
neighbourhood_latitude = merged_df.loc[4, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = merged_df.loc[4, 'Longitude'] # neighborhood longitude value

neighbourhood_name = merged_df.loc[4, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results

Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


{'meta': {'code': 200, 'requestId': '5e5b61fd211536001b61d4f8'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [250]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues[nearby_venues["categories"].str.contains(Restaurant)]

,name,categories,lat,lng
5,Impact Kitchen,Restaurant,43.656369,-79.356980
20,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843
24,Cocina Economica,Mexican Restaurant,43.654959,-79.365657
25,El Catrin,Mexican Restaurant,43.650601,-79.358920
34,Izumi,Asian Restaurant,43.649970,-79.360153
43,Flame Shack,Restaurant,43.656844,-79.358917


In [251]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [252]:
import folium
map_to = folium.Map(location=[43.656369,-79.356980], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to